In [ ]:
import io
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPool2D, Flatten

from urllib.request import urlretrieve
from zipfile import ZipFile
from PIL import Image

In [ ]:
seed = 0

In [ ]:
urlretrieve('https://raw.githubusercontent.com/prat-man/CSE-575-Statistical-Machine-Learning/main/data/spectrogram.zip', 'spectrogram.zip')

('spectrogram.zip', <http.client.HTTPMessage at 0x7f10d22b2a60>)

In [ ]:
X = []
y = []

with ZipFile('spectrogram.zip', 'r') as zip:
    for name in zip.namelist():
        genre = name.split('/')[0]
        file_name = name.split('/')[1]
        
        image_data = zip.read(name)
        image = Image.open(io.BytesIO(image_data))
        np_image = np.array(image)[35:253, 54:390, :3]
        
        X.append(np_image)
        y.append(genre)

X = np.array(X)
y = np.array(y, ndmin=2).T

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)
labels = encoder.categories_[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, stratify=y, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, stratify=y_test, random_state=seed)

In [ ]:
model_layers = [
    Input(X_train.shape[1:4]),
    
    Conv2D(32, (3, 3), activation='LeakyReLU'),
    MaxPool2D(2, 2),

    Conv2D(32, (3, 3), activation='LeakyReLU'),
    MaxPool2D(2, 2),

    Flatten(),

    Dense(48, activation='LeakyReLU'),
    Dropout(0.3),

    Dense(48, activation='LeakyReLU'),
    Dropout(0.3),

    Dense(10, activation="softmax")
]

model = Sequential(model_layers)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 216, 334, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 108, 167, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 165, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 53, 82, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 139072)            0         
                                                                 
 dense (Dense)               (None, 48)                6

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=10)

Epoch 1/20
39/70 [===============>..............] - ETA: 33s - loss: 302.7263 - accuracy: 0.1872

KeyboardInterrupt: ignored

In [ ]:
_, accuracy = model.evaluate(X_train, y_train)
print(f'Training Accuracy: {accuracy}')

In [ ]:
y_pred = model.predict(X_test)

y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
conf_m = confusion_matrix(y_true, y_pred)

sns.heatmap(conf_m, cmap='crest', annot=True)

plt.xticks(ticks = np.arange(0.5, 10, 1), labels=labels, rotation=90)
plt.yticks(ticks = np.arange(0.5, 10, 1), labels=labels,rotation=0)

plt.savefig('CNN_Confusion_Matrix.png', dpi=300)

In [ ]:
accuracy = np.trace(conf_m) / np.sum(conf_m)
print(f'Overall test accuracy = {accuracy}')

In [ ]:
accuracy_df = pd.DataFrame(columns=['accuracy'])

class_acc = conf_m.diagonal() / conf_m.sum(axis=1)
for i in range(len(labels)):
    accuracy_df.loc[labels[i]] = [round(class_acc[i], 2)]

accuracy_df

In [ ]:
model.save('CNN.h5')